## Loading the data

First we set up our imports:

In [ ]:
import yt
import numpy as np

First we load the data set, specifying both the unit length/mass/velocity, as well as the size of the bounding box (which should encapsulate all the particles in the data set)

At the end, we flatten the data into "ad" in case we want access to the raw simulation data

>This dataset is available for download at https://yt-project.org/data/GadgetDiskGalaxy.tar.gz (430 MB).

In [ ]:
fname = 'GadgetDiskGalaxy/snapshot_200.hdf5'

unit_base = {'UnitLength_in_cm'         : 3.08568e+21,
             'UnitMass_in_g'            :   1.989e+43,
             'UnitVelocity_in_cm_per_s' :      100000}

bbox_lim = 1e5 #kpc

bbox = [[-bbox_lim,bbox_lim],
        [-bbox_lim,bbox_lim],
        [-bbox_lim,bbox_lim]]
 
ds = yt.load(fname,unit_base=unit_base,bounding_box=bbox)
ds.index
ad= ds.all_data()

Let's make a projection plot to look at the entire volume

In [ ]:
px = yt.ProjectionPlot(ds, 'x', ('gas', 'density'))
px.show()

Let's print some quantities about the domain, as well as the physical properties of the simulation


In [ ]:
print ('left edge: ',ds.domain_left_edge)
print ('right edge: ',ds.domain_right_edge)
print ('center: ',ds.domain_center)

We can also see the fields that are available to query in the dataset

In [ ]:
sorted(ds.field_list)

Let's create a data object that represents the full simulation domain, and find the total mass in gas and dark matter particles contained in it:

In [ ]:
ad = ds.all_data()

# total_mass returns a list, representing the total gas and dark matter + stellar mass, respectively
print ([tm.in_units('Msun') for tm in ad.quantities.total_mass()])

Now let's say we want to zoom in on the box (since clearly the bounding we chose initially is much larger than the volume containing the gas particles!), and center on wherever the highest gas density peak is.  First, let's find this peak:

In [ ]:
density = ad["PartType0","density"]
wdens = np.where(density == np.max(density))
coordinates = ad["PartType0","Coordinates"]
center = coordinates[wdens][0]
print ('center = ',center)

Set up the box to zoom into

In [ ]:
new_box_size = ds.quan(250,'code_length')

left_edge = center - new_box_size/2
right_edge = center + new_box_size/2

print (new_box_size.in_units('Mpc'))
print (left_edge.in_units('Mpc'))
print (right_edge.in_units('Mpc'))

In [ ]:
ad2= ds.region(center=center, left_edge=left_edge, right_edge=right_edge)

Using this new data object, let's confirm that we're only looking at a subset of the domain by first calculating the total mass in gas and particles contained in the subvolume:

In [ ]:
print ([tm.in_units('Msun') for tm in ad.quantities.total_mass()])

And then by visualizing what the new zoomed region looks like

In [ ]:
px = yt.ProjectionPlot(ds, 'x', ('gas', 'density'), center=center, width=new_box_size)
px.show()

Cool - there's a disk galaxy there!